In [4]:
# -*- coding: utf-8 -*-
# Pyomo + Gurobi: Lagrangean-based (per Karuppiah & Grossmann, 2008) – single-node loop
from dataclasses import dataclass, field
from typing import Callable, List, Dict, Tuple
import math

from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
import bisect
from pyomo.opt import SolverStatus, TerminationCondition


from typing import List, Dict

In [5]:
# ========== 构造单个场景 ==========
def build_scenario_model(w: float, xL: float, xU: float, params: Dict):
    """
    返回 Pyomo 模型：
      m.x  linking 副本
      m.u  场景内部连续变量
      m.obj_expr  只包含 w*s(x)+r(u)，不含 λ
    """
    m = pyo.ConcreteModel()
    m.x = pyo.Var(bounds=(xL, xU))
    m.u = pyo.Var(bounds=(0, None))

    # ---- 示例目标 ----
    a = params.get("a", 1.0)
    m.obj_expr = 0.5 * w * (m.x**2) + a * (m.u**2)

    # ---- 示例约束 ----
    c = params.get("c", 1.0); d = params.get("d", 0.0)
    m.c1 = pyo.Constraint(expr = m.u >= c*m.x + d)

    return m


# ========== λ差分函数 ==========
def lambda_diff(n: int, lambdas: List[float]) -> float:
    """
    链式结构下： (λ_n - λ_{n-1}), 其中 λ_0=0, λ_N=0
    n ∈ {1,...,N}, lambdas 有 N-1 个元素
    """
    N = len(lambdas) + 1
    lam_n = lambdas[n-1] if 1 <= n <= N-1 else 0.0
    lam_prev = lambdas[n-2] if 2 <= n <= N-1 else 0.0
    return lam_n - lam_prev


# ========== 解一轮拉格朗日松弛 ==========
def solve_lagrangian_once(models: List[pyo.ConcreteModel], lambdas: List[float]):
    solver = pyo.SolverFactory("gurobi")
    xs, objs = [], []
    N = len(models)
    for n, m in enumerate(models, start=1):
        if hasattr(m, 'obj'):
            m.del_component(m.obj)
        lag_term = lambda_diff(n, lambdas) * m.x
        m.obj = pyo.Objective(expr = m.obj_expr + lag_term, sense=pyo.minimize)
        solver.solve(m, tee=False)
        xs.append(pyo.value(m.x))
        objs.append(pyo.value(m.obj))
    return xs, objs, sum(objs)


# ========== 次梯度更新 ==========
def subgradient_update(lambdas: List[float], xs: List[float], zUB: float, zLB: float, t: float=0.5):
    """
    g_n = x^n - x^{n+1},  n=1,...,N-1
    α = t * (zUB - zLB) / ||g||^2
    """
    N = len(xs)
    g = [xs[n-1] - xs[n] for n in range(1, N)]
    g2 = sum(gi*gi for gi in g)
    if g2 <= 1e-16:
        return lambdas[:]  # 已经很一致，不更新
    alpha = t * max(0.0, zUB - zLB) / g2
    return [lam + alpha*g[n] for n, lam in enumerate(lambdas)]


# ========== 主循环 ==========
def lagrangian_loop(models: List[pyo.ConcreteModel],
                    max_iter: int = 20,
                    zUB: float = 1e9,
                    t: float = 0.5,
                    lam0: List[float] = None):
    N = len(models)
    lambdas = lam0 if lam0 is not None else [0.0]*(N-1)
    history = []
    for k in range(max_iter):
        xs, objs, zLB = solve_lagrangian_once(models, lambdas)
        history.append((k, zLB, xs, lambdas[:]))
        print(f"iter {k:02d}: zLB={zLB:.6f}, xs={['%.4f'%x for x in xs]}, λ={['%.3f'%l for l in lambdas]}")
        lambdas = subgradient_update(lambdas, xs, zUB, zLB, t=t)
    return history

iter 00: zLB=0.123328, xs=['0.1000', '0.1926', '0.1000'], λ=['0.500', '-0.200']
iter 01: zLB=-9069419.120429, xs=['3.0000', '0.1000', '0.1000'], λ=['-3239082.069', '3239082.369']
iter 02: zLB=-3027767.648174, xs=['3.0000', '0.1000', '0.1000'], λ=['-1155753.975', '3239082.369']
iter 03: zLB=-611107.059270, xs=['3.0000', '0.1000', '0.1000'], λ=['-322422.738', '3239082.369']
iter 04: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 05: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 06: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 07: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 08: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 09: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000'], λ=['10909.757', '3239082.369']
iter 10: zLB=323908.391547, xs=['0.1000', '0.1000', '0.1000']

In [ ]:
# ========== 演示 ==========
if __name__ == "__main__":
    N = 3
    w = [1.0/N]*N
    xL, xU = 0.1, 3.0
    params = [
        {"a":1.0, "c":1.0, "d":0.0},
        {"a":0.8, "c":1.2, "d":0.1},
        {"a":1.2, "c":0.9, "d":0.2},
    ]
    models = [build_scenario_model(w[n], xL, xU, params[n]) for n in range(N)]
    # 初始 λ（N-1 个）
    lam0 = [0.5, -0.2]
    lagrangian_loop(models, max_iter=15, zUB=1e6, t=0.6, lam0=lam0)

In [ ]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return -(y-1)**2+1

v_list = [v_1, v_2]

# build model for each scenario 
m1 = ConcreteModel()
m1.y = Var(bounds=(0, 1))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y = Var(bounds=(0, 1))
m2.x2 = Var()
m2.c1 = Constraint(expr=m2.x2 == -(m2.y-1)**2)
m2.obj_expr = Expression(expr=m2.x2+1)

model_list = [m1, m2]
# lower bound and upper bound
ylb = 0
yub = 0.2